In [12]:
from transformers import RobertaTokenizer, RobertaForMaskedLM
import torch

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForMaskedLM.from_pretrained('roberta-base')

sentence = "Tom has fully ___ ___ ___ ___ ___ ___ illness."



In [44]:
tokenizer.unk_token

'<unk>'

In [18]:

def get_prediction (sent):
    
    token_ids = tokenizer.encode(sent, return_tensors='pt')
    masked_position = (token_ids.squeeze() == tokenizer.mask_token_id).nonzero()
    masked_pos = [mask.item() for mask in masked_position ]

    with torch.no_grad():
        output = model(token_ids)

    print(token_ids)
    print(output[0])

    last_hidden_state = output[0].squeeze()
    # print(last_hidden_state)
    list_of_list =[]
    for index, mask_index in enumerate(masked_pos):
        mask_hidden_state = last_hidden_state[mask_index]
        # print(mask_hidden_state)
        idx = torch.topk(mask_hidden_state, k=5, dim=0)[1]
        words = [tokenizer.decode(i.item()).strip() for i in idx]
        list_of_list.append(words)
        # print ("Mask ",index+1,"Guesses : ",words)
    
    best_guess = ""
    for j in list_of_list:
        best_guess = best_guess+" "+j[0]
        
    return best_guess


print ("Original Sentence: ",sentence)
sentence = sentence.replace("___","<mask>")
print ("Original Sentence replaced with mask: ",sentence)
print ("\n")

prediction_blanks = get_prediction(sentence)

prediction_blanks

Original Sentence:  Tom has fully <mask> <mask> <mask> <mask> <mask> <mask> illness.
Original Sentence replaced with mask:  Tom has fully <mask> <mask> <mask> <mask> <mask> <mask> illness.


tensor([[    0, 15691,    34,  1950, 50264, 50264, 50264, 50264, 50264, 50264,
          5467,     4,     2]])
tensor([[[30.8177, -4.3517, 17.8006,  ...,  0.2488,  2.9159, 11.5837],
         [ 1.6616, -4.2776,  9.5994,  ..., -5.5591, -3.4594,  0.0512],
         [-0.3035, -3.6131,  7.9517,  ..., -1.7542, -1.1053,  3.3645],
         ...,
         [-3.5634, -4.8674,  8.0275,  ..., -6.8150, -5.1172, -2.5712],
         [14.3591, -5.1044, 16.8345,  ..., -3.7020, -1.5309,  6.7080],
         [11.4474, -3.8683, 28.1128,  ..., -0.4098, -1.9798,  8.0904]]])


' recovered the the of of his'

In [42]:
def get_prediction_2(sentence):
    
    inputs = tokenizer(sentence, return_tensors="pt")
    
    print(inputs["input_ids"])
    masked_positions = [x for x in (inputs["input_ids"] == tokenizer.mask_token_id)[0].nonzero(as_tuple=True)[0]]
    print(masked_positions  )
    with torch.no_grad():
        logits = model(**inputs).logits
    
    for mask_idx in masked_positions:
        predicted_token_id = logits[0, mask_idx].argmax(axis=-1)
        print(predicted_token_id)
        inputs["input_ids"][0][mask_idx] = predicted_token_id
        print(inputs["input_ids"])
        print(tokenizer.decode(predicted_token_id))
        with torch.no_grad():
            logits = model(**inputs).logits
        # mask_word = tokenizer.convert_ids_to_tokens([inputs["input_ids"][0][mask].item()])
        # mask_word_logits = logits[0, mask]
        # mask_word_prob = torch.softmax(mask_word_logits, dim=0)[mask_word]
        # print(f"Prediction for '{mask_word[0]}': {mask_word[0]} with probability {mask_word_prob.item()}")
        # top_5 = torch.topk(mask_word_logits, 5)
        # for i in range(5):
        #     word = tokenizer.convert_ids_to_tokens([top_5.indices[i].item()])
        #     print(f"Guess {i+1}: '{word[0]}' with probability {top_5.values[i].item()}")
        # print()
        print(tokenizer.decode(inputs["input_ids"][0]))
    # print(logits)
    
get_prediction_2(sentence.replace("___","<mask>"))

tensor([[    0, 15691,    34,  1950, 50264, 50264, 50264, 50264, 50264, 50264,
          5467,     4,     2]])
[tensor(4), tensor(5), tensor(6), tensor(7), tensor(8), tensor(9)]
tensor(4609)
tensor([[    0, 15691,    34,  1950,  4609, 50264, 50264, 50264, 50264, 50264,
          5467,     4,     2]])
 recovered
<s>Tom has fully recovered<mask><mask><mask><mask><mask> illness.</s>
tensor(31)
tensor([[    0, 15691,    34,  1950,  4609,    31, 50264, 50264, 50264, 50264,
          5467,     4,     2]])
 from
<s>Tom has fully recovered from<mask><mask><mask><mask> illness.</s>
tensor(39)
tensor([[    0, 15691,    34,  1950,  4609,    31,    39, 50264, 50264, 50264,
          5467,     4,     2]])
 his
<s>Tom has fully recovered from his<mask><mask><mask> illness.</s>
tensor(251)
tensor([[    0, 15691,    34,  1950,  4609,    31,    39,   251, 50264, 50264,
          5467,     4,     2]])
 long
<s>Tom has fully recovered from his long<mask><mask> illness.</s>
tensor(12)
tensor([[    0, 1569